In [1]:
import numpy as np
import pandas as pd
import warnings,time
warnings.filterwarnings("ignore")

In [2]:
X = pd.read_csv('train_removed_all.csv')
Ytrain = pd.read_csv('Y.csv')

In [3]:
# X.columns
X.drop(['Unnamed: 0'],axis = 1,inplace = True)
print(X.shape)
X.head()

(213451, 349)


,age,tfa_year,tfa_month,tfa_day,tfa_weekday_1,tfa_weekday_2,tfa_weekday_4,tfa_weekday_5,tfa_weekday_6,tfa_weekday_7,...,act_std_,act_mean_,act_type_len_,act_type_std_,act_type_mean_,act_detail_std_,act_detail_mean_,dev_type_len_,dev_type_std_,dev_type_mean_
0,100,2009,3,19,0,0,1,0,0,0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0
1,35,2009,5,23,0,0,0,0,1,0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0
2,55,2009,6,9,0,1,0,0,0,0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0
3,35,2009,10,31,0,0,0,0,1,0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0
4,35,2009,12,8,0,1,0,0,0,0,...,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0


In [4]:
Ytrain.drop(['Unnamed: 0'],axis = 1,inplace = True)
print(Ytrain.shape)
Ytrain.head()

(213451, 1)


,type
0,NDF
1,NDF
2,US
3,other
4,US


In [5]:
# 按1/5比例拆分数据
xtrain = X.sample(frac = 1/5,axis = 0)
y_dict = {}
for i in xtrain.index:
    y_dict[i] = Ytrain.iloc[i].values[0]

df_y = pd.DataFrame(pd.Series(y_dict),columns=['type'])  # dict -> Series -> DataFrame
print(df_y.shape)
df_y.head()

(42690, 1)


,type
53660,US
128455,NDF
102555,NDF
79215,NDF
160417,other


In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
# 此函数用于把源数据集按1/k的比例分为5个子集
def spilt(x,y,k = 5,n = 5):
    Xtrain_list = []
    Ytrain_list = []
    for i in range(n):
        df_x = X.sample(frac = 1/k,axis = 0)
        y_dict = {}
        for i in df_x.index:
            y_dict[i] = Ytrain.iloc[i].values[0]

        df_y = pd.DataFrame(pd.Series(y_dict),columns=['type'])  # dict -> Series -> DataFrame
        # 标准化 X
        transfer_std = StandardScaler()
        Xtrain = transfer_std.fit_transform(df_x)
        # 编码 Y
        l_e = LabelEncoder()
        encode_label = l_e.fit_transform(df_y.values)

        Xtrain_list.append(Xtrain)
        Ytrain_list.append(encode_label)
    return Xtrain_list,Ytrain_list
    
# 拆分数据集可使用 留出法 + cross_val_score交叉验证直接拆分 -> 直接求得K折的平均值
# KFold交叉验证拆分（可观察过程）-> 先将原始数据集划分，再求K折的每个准确率再求平均
# from sklearn.model_selection import train_test_split
# x_train,x_test,y_train,y_test = train_test_split(Xtrain,Y,test_size = 0.25,random_state = 100)

In [6]:
Xtrain_list,Ytrain_list = spilt(X,Ytrain,k = 7,n = 7)

In [7]:
Ytrain_list

[array([ 7,  7,  8, ...,  7, 10,  7]),
 array([ 7,  7, 10, ...,  7,  7, 10]),
 array([ 3,  0,  7, ...,  7, 10,  7]),
 array([ 7, 10,  7, ..., 10,  7,  7]),
 array([11,  1,  7, ...,  4,  7,  7]),
 array([10, 10,  7, ...,  7,  7,  5]),
 array([10,  7, 10, ...,  7, 10,  7])]

In [8]:
from sklearn.metrics import make_scorer
from sklearn.preprocessing import LabelBinarizer

def dcg_score(y_true, y_score, k=5):
    
    """
    y_true : array, shape = [n_samples] #数据
        Ground truth (true relevance labels).
    y_score : array, shape = [n_samples, n_classes] #预测的分数
        Predicted scores.
    k : int
    """
    order = np.argsort(y_score)[::-1] #分数从高到低排序
    y_true = np.take(y_true, order[:k]) #取出前k[0,k）个分数
      
    gain = 2 ** y_true - 1   

    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain / discounts)
  

def ndcg_score(ground_truth, predictions, k=5):   

    """
    Parameters
    ----------
    ground_truth : array, shape = [n_samples]
        Ground truth (true labels represended as integers).
    predictions : array, shape = [n_samples, n_classes] 
        Predicted probabilities. 预测的概率
    k : int
        Rank.
    """
    lb = LabelBinarizer()
    lb.fit(range(len(predictions) + 1))
    T = lb.transform(ground_truth)    
    scores = []
    # Iterate over each y_true and compute the DCG score
    for y_true, y_score in zip(T, predictions):
        actual = dcg_score(y_true, y_score, k)
        best = dcg_score(y_true, y_true, k)
        score = float(actual) / float(best)
        scores.append(score)

    return np.mean(scores)

In [10]:
# DT
import numpy as np
from sklearn.tree import DecisionTreeClassifier
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import classification_report
# from sklearn.metrics import hamming_loss
from sklearn.model_selection import KFold

start = time.time()
# # 网格搜索 + 交叉验证
# DT_estimator = DecisionTreeClassifier(min_samples_leaf = 12) # class_weight='balanced'  min_samples_leaf = 12
# dt_param_dict = {'max_depth':range(10,101,10),'max_leaf_nodes':range(2,11,2)} # min_samples_leaf是最小分类数，全部数据集时需要指定12类别  'max_leaf_nodes':range(2,11,2)
# dt_estimator = GridSearchCV(DT_estimator,param_grid = dt_param_dict,cv = 5)
# dt_estimator.fit(x_train,y_train)
# dt_y_predict = dt_estimator.predict(x_test)

# dt_report = classification_report(dt_y_predict,y_test,digits=5) # 准确率 + f1...
# dt_hamming_distance = hamming_loss(y_test,dt_y_predict) # 海明距离（多分类评估指标之一）
# dt_hamming_distance_train  = hamming_loss(dt_estimator.predict(x_train),y_train)  # 训练集海明距离
# dt_acc_score = dt_estimator.score(x_test,y_test)  # 测试集准确率
# print(dt_estimator.best_params_)
# print(dt_report)
# print('--'*50)
# print('训练集海明距离为{}'.format(dt_hamming_distance_train))
# print('测试集海明距离为{}'.format(dt_hamming_distance))
# print('测试集准确率为{}'.format(dt_acc_score))
# print('--'*50)

## 直接训练，得出ndcg_score
dt_ndcg_estimator = DecisionTreeClassifier(min_samples_leaf = 12,max_depth = 10,max_leaf_nodes = 9)
dt_kf = KFold(n_splits=5,random_state = 100)

dt_train_score_final = []
dt_test_score_final = []
for i in range(5):
    dt_ndcg_train_score_list = []
    dt_ndcg_test_score_list = []
    for train_index,test_index in dt_kf.split(Xtrain_list[i],Ytrain_list[i]):
    #     print(train_index,test_index)
        X_train,X_test = Xtrain_list[i][train_index,:],Xtrain_list[i][test_index,:]
        Y_train,Y_test = Ytrain_list[i][train_index],Ytrain_list[i][test_index]

        dt_ndcg_estimator.fit(X_train,Y_train)

        dt_ndcg_train_proba = dt_ndcg_estimator.predict_proba(X_train)
        dt_ndcg_test_proba = dt_ndcg_estimator.predict_proba(X_test)

        dt_ndcg_train_score = ndcg_score(Y_train,dt_ndcg_train_proba,k = 3) # 训练集ndcg
        dt_ndcg_test_score = ndcg_score(Y_test,dt_ndcg_test_proba,k = 3) # 测试集ndcg

        dt_ndcg_train_score_list.append(dt_ndcg_train_score)
        dt_ndcg_test_score_list.append(dt_ndcg_test_score)

    dt_train_score_final.append(np.mean(dt_ndcg_train_score_list))
    dt_test_score_final.append(np.mean(dt_ndcg_test_score_list))

end = time.time()
print('DT训练集NDCG为{}s'.format(np.mean(dt_train_score_final)))
print('DT测试集NDCG为{}s'.format(np.mean(dt_test_score_final)))
print('DT共耗时{}s'.format(int(end-start)))

DT训练集NDCG为0.8088806175964844s
DT测试集NDCG为0.8085779063433396s
DT共耗时1027s


In [20]:
# 根据不同核函数的SVM
import warnings
warnings.filterwarnings("ignore")
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
TOL = 1e-4
MAX_ITER = 1000
RANDOM_STATE = 100

svm_list = {
    
    'SVM-rbf': SVC(kernel='rbf',
                   max_iter=MAX_ITER,
                   tol=TOL, random_state=RANDOM_STATE,
                   decision_function_shape='ovr',probability=True),     
    
    'SVM-poly': SVC(kernel='poly',
                   max_iter=MAX_ITER,
                   tol=TOL, random_state=RANDOM_STATE,
                       decision_function_shape='ovr',probability=True),     
    
    'SVM-linear': SVC(kernel='linear',
                      max_iter=MAX_ITER,
                      tol=TOL, 
                      random_state=RANDOM_STATE,
                      decision_function_shape='ovr',probability=True),  
    
#     'LinearSVC': LinearSVC(max_iter=MAX_ITER,
#                             tol=TOL,
#                             random_state=RANDOM_STATE,
#                             multi_class = 'ovr')  
                            
}
svm_kf = KFold(n_splits=5,random_state = 100)
svm_train_ndcg_score_list = {}
svm_test_ndcg_score_list = {}
for key,value in svm_list.items():
    svm_estimator = value
    svm_train_score_final = []
    svm_test_score_final = []
    start = time.time()
    for i in range(5):
        svm_score_train_iter = []
        svm_score_test_iter = []
#         start = time.time()
        for train_index,test_index in svm_kf.split(Xtrain_list[i],Ytrain_list[i]):
    #     print(train_index,test_index)
            X_train,X_test = Xtrain_list[i][train_index,:],Xtrain_list[i][test_index,:]
            Y_train,Y_test = Ytrain_list[i][train_index],Ytrain_list[i][test_index]

            svm_estimator.fit(X_train,Y_train)

            svm_train_ndcg_score = ndcg_score(Y_train,svm_estimator.predict_proba(X_train),k = 5)  #训练集ndcg
            svm_test_ndcg_score = ndcg_score(Y_test,svm_estimator.predict_proba(X_test),k = 5)  #测试集ndcg

            svm_score_train_iter.append(svm_train_ndcg_score)
            svm_score_test_iter.append(svm_test_ndcg_score)
        svm_train_score_final.append(np.mean(svm_score_train_iter))
        svm_test_score_final.append(np.mean(svm_score_test_iter))
    
    for i in range(len(svm_score_train_iter)):
        svm_train_ndcg_score_list[key] = np.mean(svm_train_score_final[i])
    for j in range(len(svm_score_test_iter)):
        svm_test_ndcg_score_list[key] = np.mean(svm_test_score_final[i])
    end = time.time()
    print('{}算法耗时{}s'.format(key,int(end-start)))
# print(svm_score_dict)
print('训练集分数为{}'.format(svm_train_ndcg_score_list))
print('测试集分数为{}'.format(svm_test_ndcg_score_list))

SVM-rbf算法耗时53501s
SVM-poly算法耗时48070s
SVM-linear算法耗时40195s
训练集分数为{'SVM-rbf': 0.8107632623534101, 'SVM-poly': 0.8093641861523704, 'SVM-linear': 0.8069913439305136}
测试集分数为{'SVM-rbf': 0.8068063087191353, 'SVM-poly': 0.806847143367375, 'SVM-linear': 0.8065592941589017}


In [12]:
# LR
from sklearn.linear_model import LogisticRegression
import warnings,time
from sklearn.model_selection import KFold
svm_kf = KFold(n_splits=5,random_state = 100)
iters=[300,500,700,1000]
start = time.time()
lr_train_score_final = []
lr_test_score_final = []
for i in range(5):
    lr_train_ndcg_score_list = []
    lr_test_ndcg_score_list = []
    for index,value in enumerate(iters):
        lr_estimator = LogisticRegression(C=1.0,max_iter=value, tol=1e-5, solver='newton-cg', multi_class='multinomial',random_state = 100, n_jobs = -1) # 大数据solver='newton-cg'
        lr_score_train_iter = []
        lr_score_test_iter = []
        for train_index,test_index in svm_kf.split(Xtrain_list[i],Ytrain_list[i]):
    #     print(train_index,test_index)
            X_train,X_test = Xtrain_list[i][train_index,:],Xtrain_list[i][test_index,:]
            Y_train,Y_test = Ytrain_list[i][train_index],Ytrain_list[i][test_index]

            lr_estimator.fit(X_train,Y_train)

            lr_train_ndcg_score = ndcg_score(Y_train,lr_estimator.predict_proba(X_train),k = 5)  #训练集ndcg
            lr_test_ndcg_score = ndcg_score(Y_test,lr_estimator.predict_proba(X_test),k = 5)  #测试集ndcg

            lr_score_train_iter.append(lr_train_ndcg_score) #算出每一折的值
            lr_score_test_iter.append(lr_test_ndcg_score)
        lr_train_ndcg_score_list.append(np.mean(lr_score_train_iter)) #算出每一个iter中交叉验证的平均值
        lr_test_ndcg_score_list.append(np.mean(lr_score_test_iter))
        
    lr_test_score_final.append(np.mean(lr_train_ndcg_score_list))
    lr_test_score_final.append(np.mean(lr_test_ndcg_score_list))
end = time.time()
print('训练集NDCG为{}'.format(np.mean(lr_train_score_final)))
print('测试集NDCG为{}'.format(np.mean(lr_test_score_final)))
print('LR共耗时{}s'.format(int(end-start)))

训练集NDCG为0.825012892629808
测试集NDCG为0.825012892629808
LR共耗时48313s


In [13]:
# RF,AdaBoost，GradientBoost，Bagging
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier,BaggingClassifier,ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import time
LEARNING_RATE = 0.1
N_ESTIMATORS = 100
RANDOM_STATE = 100
MAX_DEPTH = 9
tree_kf = KFold(n_splits=5,random_state = 100)
#建了一个tree字典
tree_list ={
    'RF': RandomForestClassifier(n_estimators=N_ESTIMATORS,
                                 max_depth=MAX_DEPTH,
                                 random_state=RANDOM_STATE),
    
    'AdaBoost': AdaBoostClassifier(n_estimators=N_ESTIMATORS,
                                   learning_rate=LEARNING_RATE,
                                   random_state=RANDOM_STATE),
    
    'Bagging': BaggingClassifier(n_estimators=N_ESTIMATORS,
                                 random_state=RANDOM_STATE),
    
    'ExtraTree': ExtraTreesClassifier(max_depth=MAX_DEPTH,
                                      n_estimators=N_ESTIMATORS,
                                      random_state=RANDOM_STATE),
    
    'GraBoost': GradientBoostingClassifier(learning_rate=LEARNING_RATE,
                                           max_depth=MAX_DEPTH,
                                           n_estimators=N_ESTIMATORS,
                                           random_state=RANDOM_STATE)
}
tree_train_ndcg_score_list = {}
tree_test_ndcg_score_list = {}
for key,value in tree_list.items():
    tree_estimator = value
    
    tree_train_score_final = []
    tree_test_score_final = []
    for i in range(5):
        tree_score_train_iter = []
        tree_score_test_iter = []
        start = time.time()
        for train_index,test_index in tree_kf.split(Xtrain_list[i],Ytrain_list[i]):
    #     print(train_index,test_index)
            X_train,X_test = Xtrain_list[i][train_index,:],Xtrain_list[i][test_index,:]
            Y_train,Y_test = Ytrain_list[i][train_index],Ytrain_list[i][test_index]

            tree_estimator.fit(X_train,Y_train)
#             print(X_train.shape)
#             print(Y_train.shape)
            tree_train_ndcg_score = ndcg_score(Y_train,tree_estimator.predict_proba(X_train),k = 5)  #训练集ndcg
            tree_test_ndcg_score = ndcg_score(Y_test,tree_estimator.predict_proba(X_test),k = 5)  #测试集ndcg

            tree_score_train_iter.append(tree_train_ndcg_score)
            tree_score_test_iter.append(tree_test_ndcg_score)
            
        tree_train_score_final.append(np.mean(tree_score_train_iter))
        tree_test_score_final.append(np.mean(tree_score_test_iter))
    for i in range(len(tree_score_train_iter)):
        tree_train_ndcg_score_list[key] = np.mean(tree_train_score_final[i])
    for j in range(len(tree_score_test_iter)):
        tree_test_ndcg_score_list[key] = np.mean(tree_test_score_final[i])
    end = time.time()
    print('{}算法耗时{}s'.format(key,int(end-start)))
print('训练集分数为{}'.format(tree_train_ndcg_score_list))
print('测试集分数为{}'.format(tree_test_ndcg_score_list))

RF算法耗时255s
AdaBoost算法耗时475s
Bagging算法耗时2187s
ExtraTree算法耗时267s
GraBoost算法耗时23726s
训练集分数为{'RF': 0.8255420441904817, 'AdaBoost': 0.8040145522393344, 'Bagging': 0.9953975722887847, 'ExtraTree': 0.8187004672345214, 'GraBoost': 0.9493455091377362}
测试集分数为{'RF': 0.8093385432841202, 'AdaBoost': 0.8035797764349348, 'Bagging': 0.8049099222978253, 'ExtraTree': 0.8056483837874673, 'GraBoost': 0.810463947869908}


In [25]:
print('训练集分数为{}'.format(tree_train_ndcg_score_list))
print('测试集分数为{}'.format(tree_test_ndcg_score_list))

训练集分数为{'RF': 0.8309136097047173, 'AdaBoost': 0.8393518268629461, 'Bagging': 0.9591210072671774, 'ExtraTree': 0.8287156479371033, 'GraBoost': 0.9389081113987616}
测试集分数为{'RF': 0.8366363503737687, 'AdaBoost': 0.8512367478668409, 'Bagging': 0.8303176891925171, 'ExtraTree': 0.8369002700153513, 'GraBoost': 0.8285280799796116}


In [14]:
import xgboost as xgb
import time
from sklearn.model_selection import KFold

def customized_eval(preds, dtrain):
    labels = dtrain.get_label()
    top = []
    for i in range(preds.shape[0]):
        top.append(np.argsort(preds[i])[::-1][:5])
    mat = np.reshape(np.repeat(labels,np.shape(top)[1]) == np.array(top).ravel(),np.array(top).shape).astype(int)
    score = np.mean(np.sum(mat/np.log2(np.arange(2, mat.shape[1] + 2)),axis = 1))
    return 'ndcg5', score
# xgboost parameters

NUM_XGB = 200
RANDOM_STATE = 100

params = {}
params['colsample_bytree'] = 0.6
params['max_depth'] = 6
params['subsample'] = 0.8
params['eta'] = 0.3
params['seed'] = RANDOM_STATE
params['num_class'] = 12
params['objective'] = 'multi:softprob'   # output the probability instead of class. 


start = time.time()

kf = KFold(n_splits = 5, random_state=RANDOM_STATE)

k_ndcg = 5
train_score_final = []
cv_score_final = []
for i in range(5):
    train_score_iter = []
    cv_score_iter = []
    for train_index, test_index in kf.split(Xtrain_list[i], Ytrain_list[i]):

        X_train, X_test = Xtrain_list[i][train_index, :], Xtrain_list[i][test_index, :]
        y_train, y_test = Ytrain_list[i][train_index], Ytrain_list[i][test_index]

        train_xgb = xgb.DMatrix(X_train, label= y_train)
        test_xgb = xgb.DMatrix(X_test, label = y_test)

        watchlist = [ (train_xgb,'train'), (test_xgb, 'test') ]

        bst = xgb.train(params, 
                         train_xgb,
                         NUM_XGB,
                         watchlist,
                         feval = customized_eval,
                         verbose_eval = 3,
                         early_stopping_rounds = 5)


        #bst = xgb.train( params, dtrain, num_round, evallist )

        y_pred = np.array(bst.predict(test_xgb))
        y_pred_train = np.array(bst.predict(train_xgb))
        train_ndcg_score = ndcg_score(y_train, y_pred_train , k = k_ndcg)
        cv_ndcg_score = ndcg_score(y_test, y_pred, k=k_ndcg)

        train_score_iter.append(train_ndcg_score)
        cv_score_iter.append(cv_ndcg_score)
        
    train_score_final.append(np.mean(train_score_iter))
    cv_score_final.append(np.mean(cv_score_iter))

train_score_xgb = np.mean(train_score_final)
cv_score_xgb = np.mean(cv_score_final)
end = time.time()

print ("\nThe training score is: {}".format(train_score_xgb))
print ("The cv score is: {}\n".format(cv_score_xgb))
print('原生xgboost共耗时{}s'.format(int(end-start)))

[0]	train-merror:0.364434	test-merror:0.367273	train-ndcg5:0.827848	test-ndcg5:0.820532
Multiple eval metrics have been passed: 'test-ndcg5' will be used for early stopping.

Will train until test-ndcg5 hasn't improved in 5 rounds.
[3]	train-merror:0.352628	test-merror:0.36301	train-ndcg5:0.835724	test-ndcg5:0.824556
Stopping. Best iteration:
[0]	train-merror:0.364434	test-merror:0.367273	train-ndcg5:0.827848	test-ndcg5:0.820532

[0]	train-merror:0.363204	test-merror:0.378423	train-ndcg5:0.827393	test-ndcg5:0.818079
Multiple eval metrics have been passed: 'test-ndcg5' will be used for early stopping.

Will train until test-ndcg5 hasn't improved in 5 rounds.
[3]	train-merror:0.352546	test-merror:0.372356	train-ndcg5:0.835046	test-ndcg5:0.822141
Stopping. Best iteration:
[0]	train-merror:0.363204	test-merror:0.378423	train-ndcg5:0.827393	test-ndcg5:0.818079

[0]	train-merror:0.366812	test-merror:0.361207	train-ndcg5:0.82742	test-ndcg5:0.823166
Multiple eval metrics have been passed: 'tes

Stopping. Best iteration:
[0]	train-merror:0.396393	test-merror:0.395867	train-ndcg5:0.816656	test-ndcg5:0.812312

[0]	train-merror:0.390736	test-merror:0.408167	train-ndcg5:0.818517	test-ndcg5:0.807275
Multiple eval metrics have been passed: 'test-ndcg5' will be used for early stopping.

Will train until test-ndcg5 hasn't improved in 5 rounds.
[3]	train-merror:0.356385	test-merror:0.372581	train-ndcg5:0.835308	test-ndcg5:0.821435
Stopping. Best iteration:
[0]	train-merror:0.390736	test-merror:0.408167	train-ndcg5:0.818517	test-ndcg5:0.807275

[0]	train-merror:0.368328	test-merror:0.37334	train-ndcg5:0.825098	test-ndcg5:0.819853
Multiple eval metrics have been passed: 'test-ndcg5' will be used for early stopping.

Will train until test-ndcg5 hasn't improved in 5 rounds.
[3]	train-merror:0.357711	test-merror:0.366617	train-ndcg5:0.831603	test-ndcg5:0.82498
Stopping. Best iteration:
[0]	train-merror:0.368328	test-merror:0.37334	train-ndcg5:0.825098	test-ndcg5:0.819853

[0]	train-merror:0

In [16]:
#  Sklearn -> XGB
from xgboost.sklearn import XGBClassifier
start = time.time()
xgb_ndcg_estimator = XGBClassifier(booster = 'gbtree',silent = False,n_estimators = 100,  # nthread = -1 用全部CPU
                   subsample = 0.8,reg_alpha = 1.0, reg_lambda = 1.0,
                   objective = 'multi:softprob',num_class = 12 ,colsample_bytree = 0.6,
                   seed = 1,random_state = 100,n_jobs = 2,max_depth = 6,learning_rate = 0.3)
xgb_kf = KFold(n_splits = 3,random_state = 100)

xgb_train_score_final = []
xgb_test_score_final = []
for i in range(5):
    xgb_ndcg_train_score_list = []
    xgb_ndcg_test_score_list = []
    for train_index,test_index in xgb_kf.split(Xtrain_list[i],Ytrain_list[i]):
    #     print(train_index,test_index)
        X_train,X_test = Xtrain_list[i][train_index,:],Xtrain_list[i][test_index,:]
        Y_train,Y_test = Ytrain_list[i][train_index],Ytrain_list[i][test_index]

        xgb_ndcg_estimator.fit(X_train,Y_train)

        xgb_ndcg_train_proba = xgb_ndcg_estimator.predict_proba(X_train)
        xgb_ndcg_test_proba = xgb_ndcg_estimator.predict_proba(X_test)

        xgb_ndcg_train_score = ndcg_score(Y_train,xgb_ndcg_train_proba,k = 3) # 训练集ndcg
        xgb_ndcg_test_score = ndcg_score(Y_test,xgb_ndcg_test_proba,k = 3) # 测试集ndcg

        xgb_ndcg_train_score_list.append(xgb_ndcg_train_score)
        xgb_ndcg_test_score_list.append(xgb_ndcg_test_score)
        
    xgb_train_score_final.append(np.mean(xgb_ndcg_train_score_list))
    xgb_test_score_final.append(np.mean(xgb_ndcg_test_score_list))

end = time.time()
print('训练集NDCG为{}'.format(np.mean(xgb_train_score_final)))
print('测试集NDCG为{}'.format(np.mean(xgb_test_score_final)))
print('xgboost.Sklearn共耗时{}s'.format(int(end-start)))

训练集NDCG为0.8993966455465167
测试集NDCG为0.804885004183881
xgboost.Sklearn共耗时5971s


In [17]:
# KNN
from sklearn.neighbors import KNeighborsClassifier

knn_ndcg_estimator = KNeighborsClassifier(n_neighbors = 12)

start = time.time()
knn_kf = KFold(n_splits = 3,random_state = 100)

knn_train_score_final = []
knn_test_score_final = []
for i in range(5):
    knn_ndcg_train_score_list = []
    knn_ndcg_test_score_list = []
    for train_index,test_index in knn_kf.split(Xtrain_list[i],Ytrain_list[i]):
    #     print(train_index,test_index)
        X_train,X_test = Xtrain_list[i][train_index,:],Xtrain_list[i][test_index,:]
        Y_train,Y_test = Ytrain_list[i][train_index],Ytrain_list[i][test_index]

        knn_ndcg_estimator.fit(X_train,Y_train)

        knn_ndcg_train_proba = knn_ndcg_estimator.predict_proba(X_train)
        knn_ndcg_test_proba = knn_ndcg_estimator.predict_proba(X_test)

        knn_ndcg_train_score = ndcg_score(Y_train,knn_ndcg_train_proba,k = 3) # 训练集ndcg
        knn_ndcg_test_score = ndcg_score(Y_test,knn_ndcg_test_proba,k = 3) # 测试集ndcg

        knn_ndcg_train_score_list.append(knn_ndcg_train_score)
        knn_ndcg_test_score_list.append(knn_ndcg_test_score)
        
    knn_train_score_final.append(np.mean(knn_ndcg_train_score_list))
    knn_test_score_final.append(np.mean(knn_ndcg_test_score_list))

end = time.time()
print('训练集NDCG为{}'.format(np.mean(knn_train_score_final)))
print('测试集NDCG为{}'.format(np.mean(knn_test_score_final)))
print('KNN共耗时{}s'.format(int(end-start)))

训练集NDCG为0.8270891777031437
测试集NDCG为0.7756198683454918
KNN共耗时5439s
